<a href="https://colab.research.google.com/github/pachterlab/CP_2023/blob/main/notebooks/celltypePred_Fig4a/kimetal_smartseq_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os


from tqdm import tnrange, tqdm_notebook
def download_file(doi,ext):
	url = 'https://api.datacite.org/dois/'+doi+'/media'
	r = requests.get(url).json()
	netcdf_url = r['data'][0]['attributes']['url']
	r = requests.get(netcdf_url,stream=True)
	#Set file name
	fname = doi.split('/')[-1]+ext
	#Download file with progress bar
	if r.status_code == 403:
		print("File Unavailable")
	if 'content-length' not in r.headers:
		print("Did not get file")
	else:
		with open(fname, 'wb') as f:
			total_length = int(r.headers.get('content-length'))
			pbar = tnrange(int(total_length/1024), unit="B")
			for chunk in r.iter_content(chunk_size=1024):
				if chunk:
					pbar.update()
					f.write(chunk)
		return fname


#SMART-seq VMH data
#metadata.csv
download_file('10.22002/D1.2067','.gz')

#smartseq.mtx (log counts)
download_file('10.22002/D1.2071','.gz')

#smartseq.mtx (raw counts)
download_file('10.22002/D1.2070','.gz')

#gene names
download_file('10.22002/D1.2068','.gz')


os.system("gunzip *.gz")

os.system("mv D1.2067 metadata.csv")
os.system("mv D1.2071 smartseq.mtx")

os.system("mv D1.2070 smartseqCount.mtx")
os.system("mv D1.2068 gene_names.npy")

<ipython-input-1-af385812caee>:21: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  pbar = tnrange(int(total_length/1024), unit="B")


  0%|          | 0/64 [00:00<?, ?B/s]

  0%|          | 0/26700 [00:00<?, ?B/s]

  0%|          | 0/16654 [00:00<?, ?B/s]

  0%|          | 0/11 [00:00<?, ?B/s]

0

In [ ]:
!git clone https://github.com/pachterlab/CP_2023.git

Cloning into 'CP_2023'...
remote: Enumerating objects: 1299, done.
remote: Counting objects: 100% (1299/1299), done.
remote: Compressing objects: 100% (536/536), done.
remote: Total 1299 (delta 742), reused 1283 (delta 730), pack-reused 0
Receiving objects: 100% (1299/1299), 436.00 MiB | 20.26 MiB/s, done.
Resolving deltas: 100% (742/742), done.
Updating files: 100% (138/138), done.


In [ ]:
!pwd

/content


In [ ]:
!pip3 install --quiet torch
!pip3 install --quiet anndata
!pip3 install --quiet matplotlib
!pip3 install --quiet scikit-learn
!pip3 install --quiet torchsummary
!pip install --quiet scanpy
# !pip3 install --quiet scvi-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
%cd /content/CP_2023/scripts

/content/CP_2023/scripts


## **Install Packages**

In [ ]:
import anndata
import pandas as pd
import numpy as np

import random

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.neighbors import NeighborhoodComponentsAnalysis, NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
import torch
import time
import scanpy as sc
import seaborn as sns
import umap
from scipy import stats
import scipy.io as sio
sns.set_style('white')

## **Import Data**

In [ ]:
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['axes.linewidth'] = 0.1

state = 42
ndims = 2

data_path = '../..'

pcs = 50
pcs2 = 40
n_latent = 50

In [ ]:
count_mat = sio.mmread(data_path+'/smartseq.mtx')
count_mat.shape

(3850, 1999)

In [ ]:
raw_count_mat = sio.mmread(data_path+'/smartseqCount.mtx')
raw_count_mat.shape

(3850, 1999)

In [ ]:
#Center and scale data
scaled_mat = scale(count_mat)

In [ ]:
meta = pd.read_csv(data_path+'/metadata.csv',index_col = 0)
meta.head()

,sample_name,smartseq_cluster_id,smartseq_cluster,sex_label,smartseq_cluster_color,medical_cond_label,cell_counts,n_genes,percent_mito,pass_count_filter,pass_mito_filter
SM-GE4R2_S062_E1-50,SM-GE4R2_S062_E1-50,46,Nr5a1_9|11 Rorb,M,#6C39A6,behavior - none,964371.56,9772,0.0,True,True
SM-GE4SI_S356_E1-50,SM-GE4SI_S356_E1-50,46,Nr5a1_9|11 Rorb,M,#6C39A6,behavior - none,973072.06,8253,0.0,True,True
SM-GE4SI_S172_E1-50,SM-GE4SI_S172_E1-50,46,Nr5a1_9|11 Rorb,M,#6C39A6,behavior - none,977132.60,9394,0.0,True,True
LS-15034_S07_E1-50,LS-15034_S07_E1-50,42,Nr5a1_4|7 Glipr1,M,#763BFF,behavior - none,948605.44,10643,0.0,True,True
LS-15034_S28_E1-50,LS-15034_S28_E1-50,42,Nr5a1_4|7 Glipr1,F,#763BFF,behavior - none,951803.00,10550,0.0,True,True


In [ ]:
clusters = np.unique(meta['smartseq_cluster'].values)
map_dict = {}
for i, c in enumerate(clusters):
  map_dict[c] = i
new_labs = [map_dict[c] for c in meta['smartseq_cluster'].values]

In [ ]:
adata = anndata.AnnData(count_mat, obs = meta)
adata.X = np.nan_to_num(adata.X)

In [ ]:
adata2 = anndata.AnnData(raw_count_mat, obs = meta)
adata2.X = np.nan_to_num(adata2.X)

In [ ]:
def knn_infer(embd_space, labeled_idx, labeled_lab, unlabeled_idx,n_neighbors=50):
	"""
	Predicts the labels of unlabeled data in the embedded space with KNN.
	Parameters
	----------
	embd_space : ndarray (n_samples, embedding_dim)
		Each sample is described by the features in the embedded space.
		Contains all samples, both labeled and unlabeled.
	labeled_idx : list
		Indices of the labeled samples (used for training the classifier).
	labeled_lab : ndarray (n_labeled_samples)
		Labels of the labeled samples.
	unlabeled_idx : list
		Indices of the unlabeled samples.
	Returns
	-------
	pred_lab : ndarray (n_unlabeled_samples)
		Inferred labels of the unlabeled samples.
	"""

	# obtain labeled data and unlabled data from indices
	labeled_samp = embd_space[labeled_idx, :]
	unlabeled_samp = embd_space[unlabeled_idx, :]

	from sklearn.neighbors import KNeighborsClassifier

	knn = KNeighborsClassifier(n_neighbors=n_neighbors)
	knn.fit(labeled_samp, labeled_lab)

	pred_lab = knn.predict(unlabeled_samp)
	return pred_lab


### **Test Cell Type Prediction Capabilities for Multiple Benchmarks**
Also test accuracy for sex labels

In [ ]:
lab1 = list(meta.smartseq_cluster)
lab2 = list(meta.sex_label)
lab3 = list(meta.medical_cond_label)

lab4 = list(meta.smartseq_cluster_id)


allLabs = np.array([lab1])
allLabs2 = np.array([lab1,lab2])

nanLabs = np.array([[np.nan]*len(lab1)])

#Shuffled labels for over-fitting check
shuff_lab1 = random.sample(lab1, len(lab1))
shuff_lab2 = random.sample(lab2, len(lab2))
shuff_allLabs = np.array([shuff_lab1,shuff_lab2])

clus_colors = list(pd.unique(meta.smartseq_cluster_color))

sex_colors = ['#abacb7','#F8C471']

First test 2D space predictions (t-SNE, UMAP, UMAP-Supervised)

In [ ]:
ndims = 2
acc_score_2D = []

for i in range(3):
  reducer = umap.UMAP(n_components = ndims)
  tsne = TSNE(n_components = ndims)


  tsvd = TruncatedSVD(n_components=pcs)
  x_pca = tsvd.fit_transform(scaled_mat)

  pcaUMAP = reducer.fit_transform(x_pca)
  pcaTSNE = tsne.fit_transform(x_pca)

  #Partially labeled UMAP

  labels = np.array([lab4]).copy().astype(np.int8)
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False) #0.7 for training fraction
  #Set 30% to no label (nan)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1

  pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

  preds = knn_infer(pcaUMAPLab, train_inds, adata.obs.smartseq_cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.smartseq_cluster_id.values[unlab_inds], preds)
  acc_score_2D.append(acc)

  preds = knn_infer(pcaUMAP, train_inds, adata.obs.smartseq_cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.smartseq_cluster_id.values[unlab_inds], preds)
  acc_score_2D.append(acc)

  preds = knn_infer(pcaTSNE, train_inds, adata.obs.smartseq_cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.smartseq_cluster_id.values[unlab_inds], preds)
  acc_score_2D.append(acc)

In [ ]:
 print(acc_score_2D)

[0.9012987012987013, 0.8735930735930736, 0.8441558441558441, 0.8917748917748918, 0.8562770562770563, 0.8554112554112554, 0.8640692640692641, 0.8545454545454545, 0.8346320346320346]


Test accuracy in PCA 40D --> 2D space

In [ ]:
ndims = 2
acc_score_40_2D = []

for i in range(3):
  reducer = umap.UMAP(n_components = ndims)
  tsne = TSNE(n_components = ndims)


  tsvd = TruncatedSVD(n_components=pcs2)
  x_pca = tsvd.fit_transform(scaled_mat)

  pcaUMAP = reducer.fit_transform(x_pca)
  pcaTSNE = tsne.fit_transform(x_pca)

  #Partially labeled UMAP

  labels = np.array([lab4]).copy().astype(np.int8)
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False) #0.7 for training fraction
  #Set 30% to no label (nan)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1

  pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

  preds = knn_infer(pcaUMAPLab, train_inds, adata.obs.smartseq_cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.smartseq_cluster_id.values[unlab_inds], preds)
  acc_score_40_2D.append(acc)

  preds = knn_infer(pcaUMAP, train_inds, adata.obs.smartseq_cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.smartseq_cluster_id.values[unlab_inds], preds)
  acc_score_40_2D.append(acc)

  preds = knn_infer(pcaTSNE, train_inds, adata.obs.smartseq_cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.smartseq_cluster_id.values[unlab_inds], preds)
  acc_score_40_2D.append(acc)

In [ ]:
print(acc_score_40_2D)

In [ ]:
# from tqdm import tqdm #Need to intialize to stop tqdm errors
# tqdm(disable=True, total=0)

PCA 50D

In [ ]:
# Reconstruction loss only
acc_scorePCA = []
acc_scorePCA2 = []

for i in range(3):

  tsvd = TruncatedSVD(n_components=pcs)
  x_pca = tsvd.fit_transform(scaled_mat)

  labels = np.array([lab1])
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = np.nan

  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs.smartseq_cluster.values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs.smartseq_cluster.values[unlabeled_idx], preds)
  acc_scorePCA.append(acc)

  preds2 = knn_infer(x_pca, train_inds, adata.obs.sex_label.values[train_inds], unlabeled_idx)
  acc2 = accuracy_score(adata.obs.sex_label.values[unlabeled_idx], preds2)
  acc_scorePCA2.append(acc2)
  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

In [ ]:
print(acc_scorePCA)
print(acc_scorePCA2)

[0.8666666666666667, 0.8961038961038961, 0.8805194805194805]
[0.8917748917748918, 0.9038961038961039, 0.8813852813852814]


PCA 40D

In [ ]:
# Reconstruction loss only
acc_scorePCA40D = []

for i in range(3):

  tsvd = TruncatedSVD(n_components=pcs2)
  x_pca = tsvd.fit_transform(scaled_mat)

  labels = np.array([lab1])
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = np.nan

  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs.smartseq_cluster.values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs.smartseq_cluster.values[unlabeled_idx], preds)
  acc_scorePCA40D.append(acc)


  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

In [ ]:
print(acc_scorePCA40D)

PCA 100D

In [ ]:
# Reconstruction loss only
acc_scorePCA100D = []


for i in range(3):

  tsvd = TruncatedSVD(n_components=100)
  x_pca = tsvd.fit_transform(scaled_mat)

  labels = np.array([lab1])
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = np.nan

  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs.smartseq_cluster.values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs.smartseq_cluster.values[unlabeled_idx], preds)
  acc_scorePCA100D.append(acc)


In [ ]:
print(acc_scorePCA100D)

[0.8692640692640693, 0.8597402597402597, 0.8857142857142857]


### **Save Analysis Output**

In [ ]:
vals = pd.DataFrame()

vals['Accuracy'] = acc_scorePCA + acc_scorePCA2 + acc_score_2D + acc_scorePCA100D +acc_score_40_2D +acc_scorePCA40D #+ netAE_score + netAE_score2

In [ ]:
vals['Embed'] =  ['PCA 50D']*3 +  ['PCA 50D']*3 + ['PCA 50D UMAP Sup.','PCA 50D UMAP','PCA 50D t-SNE']*3 + ['PCA 100D']*3 +['PCA 40D UMAP Sup.','PCA 40D UMAP','PCA 40D t-SNE']*3 + ['PCA 40D']*3

In [ ]:
vals['Label'] = ['CellType1']*3 + ['Gender2']*3 + ['CellType1']*12+['CellType1']*12

In [ ]:
vals


,Accuracy,Embed,Label
0,0.883117,PCA 50D,CellType1
1,0.880519,PCA 50D,CellType1
2,0.893506,PCA 50D,CellType1
3,0.908225,PCA 50D,Gender2
4,0.899567,PCA 50D,Gender2
5,0.912554,PCA 50D,Gender2
6,0.896970,PCA 50D UMAP Sup.,CellType1
7,0.866667,PCA 50D UMAP,CellType1
8,0.844156,PCA 50D t-SNE,CellType1
9,0.894372,PCA 50D UMAP Sup.,CellType1


In [ ]:
from google.colab import files

vals.to_csv('allSmartSeqPreds_22.csv')
files.download('allSmartSeqPreds_22.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0.9201716738197425